In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
from shapely import geometry
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

from phase_o_matic.download import download_era
from phase_o_matic.preprocess import get_vapor_partial_pressure, convert_pressure_to_pascals, interpolate_to_heights
from phase_o_matic.phase_delay import calculate_refractive_indexes

In [ ]:
import pyaps3 as pa
import os
import matplotlib.pyplot as plt

data_dir = '/Users/zachkeskinen/Documents/phase-o-matic/PyAPS/tests/data'
# read geometry files
print('read ISCE geometry files: hgt/los/lat/lon.rdr')
dem = pa.utils.read_data(os.path.join(data_dir, 'hgt.rdr'))
inc = pa.utils.read_data(os.path.join(data_dir, 'los.rdr'), dname='inc')
lat = pa.utils.read_data(os.path.join(data_dir, 'lat.rdr'))
lon = pa.utils.read_data(os.path.join(data_dir, 'lon.rdr'))

# calculate
print('calculate tropospheric delay from GRB files...')
print('------------------------------------------------')
grb_file1 = os.path.join(data_dir, 'ERA5/ERA5_N30_N40_E120_E140_20101017_14.grb')
grb_file2 = os.path.join(data_dir, 'ERA5/ERA5_N30_N40_E120_E140_20110117_14.grb')
obj1 = pa.PyAPS(grb_file1, dem=dem, inc=inc, lat=lat, lon=lon, grib='ERA5', verb=True)
obj2 = pa.PyAPS(grb_file2, dem=dem, inc=inc, lat=lat, lon=lon, grib='ERA5', verb=True)
phs = obj2.getdelay(wvl = 0.238403545) - obj1.getdelay(wvl = 0.238403545)

# plot
date12 = '{}_{}'.format(grb_file1.split('_')[-2], grb_file2.split('_')[-2])
titles = [f'Tropospheric delay\n{date12}', 'Height']
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=[10, 6])
for ax, data, title in zip(axs, [phs, dem], titles):
    im = ax.imshow(data, interpolation='nearest')
    # axis format
    ax.invert_yaxis()
    ax.set_title(title)
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('meter')
fig.tight_layout()

print('------------------------------------------------')
print('Passed tropospheric delay calculation from ERA5.')
print('------------------------------------------------')
plt.show()

In [ ]:
xr.DataArray(obj1.dem, coords = [obj1.lat[:,0], obj1.lon[0, :]], dims = ['latitude', 'longitude'])

In [ ]:
# interpolate delays to uavsar
# SNWE: (33.85, 30.05, 129.05, 132.45)
# N30_N40_E120_E140
geom = geometry.box(129.05, 30.05, 132.45, 33.85)
time1 = pd.to_datetime('2010-10-17T14:00')
time2 = pd.to_datetime('2011-01-17T14:00')
time_DAs = []
max_alt = None
for time in [time1, time2]:
    fp = download_era(date = time, out_dir = '../data/', subset = geom, LOS = False, humid_param='specific_humidity')
    # fp = f"/Users/zachkeskinen/Documents/phase-o-matic/data/ERA5_{time.strftime('%Y-%m-%dT%H')}:00_-116.31_43.59_-114.90_44.46.nc"
    era = xr.open_dataset(fp) #'../data/ERA5_2021-01-15T12:00_-116.31_43.59_-114.90_44.46.nc'
    era = convert_pressure_to_pascals(era)
    era = get_vapor_partial_pressure(era)
    # max_alt = (geom['dem'] + np.abs(geom['lkv_z'])).max() if max_alt is None else max_alt
    ds = interpolate_to_heights(era, max_alt = 42000)
    ds = calculate_refractive_indexes(ds)
    # new stuff below
    # ds = ds.interp_like(geom, method = 'linear', kwargs={"fill_value": "extrapolate"})
    # ds = ds.interp(height = np.round(np.arange(geom['dem'].min()-2, geom['dem'].max()+2)), method = 'linear')
    time_DAs.append(ds)
ds = xr.concat(time_DAs, dim = 'time')

In [ ]:
# import pygrib
# lvls = np.array([1, 2, 3, 5, 7, 10, 20, 30, 50, 70, 100, 125, 150, 175, 
#                      200, 225, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 775, 
#                      800, 825, 850, 875, 900, 925, 950, 975, 1000])
# nlvls = len(lvls)
# minlat, maxlat, minlon, maxlon = obj1.minlat, obj1.maxlat, obj1.minlon, obj1.maxlon

# from era import initconst
# cdic = initconst()
# humidity = obj1.humidity
# alpha = cdic['Rv']/cdic['Rd']
# gphind = np.arange(nlvls)*12+1
# verbose = True

# grbs = pygrib.open(obj1.gfile)
# grbs.seek(gphind[0])
# grb=grbs.read(1)[0]
# lats, lons = grb.latlons()
# g = cdic['g']
# mask = (lats > minlat) & (lats < maxlat) & (lons > minlon) & (lons < maxlon)
# [ii,jj] = np.where(mask == True)
# del mask
# latlist = lats[ii,jj]
# lonlist = lons[ii,jj]
# nstn = len(ii)

# ####Create arrays for 3D storage
# gph = np.zeros((nlvls, nstn))     #Potential height
# tmp = gph.copy()                  #Temperature
# vpr = gph.copy()                  #Vapor pressure
# if verbose:
#     print('Number of stations:', nstn)

# lvls = 100.0*lvls                 #Conversion to absolute pressure
# for i in range(nlvls):
#     grbs.seek(gphind[i])          #Reading potential height.
#     grb = grbs.read(3)
#     val = grb[0].values
#     gph[i,:] = val[ii,jj]/g

#     val = grb[1].values           #Reading temperature
#     temp = val[ii,jj]
#     tmp[i,:] = temp

#     if humidity in ('R'):
#         esat = cc_era(temp,cdic)      
#         grbs.seek(gphind[i]+6)
#         grb = grbs.read(1)
#         val = grb[0].values
#         temp = val[ii,jj]/100.0
#         vpr[i,:] = temp*esat
            
#     elif humidity in ('Q'):
#         print(grb[1])
#         val = grb[1].values       #Specific humidity
#         print(lvls[i])
#         temp = val[ii,jj]
#         print(temp[0])
#         vpr[i,:] = temp*lvls[i]*alpha/(1+(alpha - 1)*temp)
#         print(vpr[i, 0])
#         print('--')


In [ ]:
from pyaps3.processor import PTV2del
dry, wet = PTV2del(obj1.Pi,obj1.Ti,obj1.Vi,obj1.hgt,obj1.dict,verbose=True)

In [ ]:
ds['N_wet'].isel(time = 0, longitude = 0, latitude = 0).plot()

In [ ]:
plt.plot(wet[0,0])

In [ ]:
from pyaps3.era import get_ecmwf
lvls,latlist,lonlist,gph,tmp,vpr,humid = get_ecmwf(obj1.model, obj1.gfile, obj1.minlat, obj1.maxlat, obj1.minlon, obj1.maxlon, obj1.dict, extra= True)

In [ ]:
q = xr.DataArray(humid.reshape(*latlist.shape, *lvls.shape), coords = [latlist[:,0], lonlist[0, :], lvls], dims = ['latitude', 'longitude','level']).to_dataset(name = 'q')

In [ ]:
era.sel(latitude = q.isel(latitude = 0).latitude, method='nearest')['q'].plot()

In [ ]:
q.transpose()['q'].isel(latitude = 0, ).data #() #get_vapor_partial_pressure(q)

In [ ]:
fig, axes = plt.subplots(3)
lat_i = 0
lon_i = 0

ax = axes[0]
ds['temperature'].isel(time = 0).sel(latitude= obj1.latlist[lat_i,0], longitude = obj1.lonlist[0,lon_i], method = 'nearest').plot(ax = ax)
ax.plot(obj1.hgt, obj1.Ti[lat_i,lon_i])

ax = axes[1]
ds['air_pressure'].isel(time = 0).sel(latitude= obj1.latlist[lat_i,0], longitude = obj1.lonlist[0,lon_i], method = 'nearest').plot(ax = ax)
ax.plot(obj1.hgt, obj1.Pi[lat_i,lon_i])

ax = axes[2]
ds['vapor_pressure'].isel(time = 0).sel(latitude= obj1.latlist[lat_i,0], longitude = obj1.lonlist[0,lon_i], method = 'nearest').plot(ax = ax)
ax.plot(obj1.hgt, obj1.Vi[lat_i,lon_i])

plt.yscale('log')

plt.tight_layout()